In [1]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files

from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). Note the `repo_release_id` and `repo_corpus_id` parameters; these tell the module which ngram corpus and release to download.

In [2]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng-fiction',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True
)

Start Time:                2025-02-03 15:56:10.774053

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/eng-fiction-1-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download
File index range:          0 to 0
File URLs available:       1
File URLs to use:          1
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/1-00000-of-00001.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng-fiction/1-00000-of-00001.gz
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-02-03 15:56:42.634337
Total runtime:             0:00:31.860284



### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [3]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-03 15:56:42.642384

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download/1-00000-of-00001.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/1download/1-00000-of-00001.txt.lz4
Ngram size:                1
Ngram type:                tagged
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-02-03 15:58:35.101897
Total runtime:             0:01:52.459513



### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

In [4]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-03 15:58:35.105719

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/2convert/1-00000-of-00001.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-02-03 15:59:26.634457
Total runtime:             0:00:51.528738



### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

In [5]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-02-03 15:59:26.637752

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize
File index range:          0 to 0
Files available:           1
Files to use:              1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/3lowercase/1-00000-of-00001.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/1 [00:00<?, ?files/s]


End Time:                  2025-02-03 16:00:29.218294
Total runtime:             0:01:02.580542



### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

In [6]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                   2025-02-03 16:02:50.906800

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter
File index range:             0 to 0
Files available:              1
Files to use:                 1
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize/1-00000-of-00001.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/4lemmatize/1-00000-of-00001.jsonl.lz4
Ngram size:                   1
Number of workers:            48
Compress output files:        True
Overwrite existing files:     False
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:         

Filtering:   0%|          | 0/1 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              2704 
Short-word tokens:            9544 
Tokens with numerals:         239217 
Tokens with non-alpha chars:  162479
Out-of-vocab tokens:          0
Entire ngrams:                413944 

End Time:                  2025-02-03 16:03:43.951387
Total runtime:             0:00:53.044587



### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

In [2]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=True,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-02-03 16:19:19.362408

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4
Files available:           1
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter/1-00000-of-00001.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/5filter/1-00000-of-00001.jsonl.lz4
Files to use:              1
Ngram size:                1
Number of workers:         12
Compress output files:     True
Overwrite existing files:  True
Sort key:                  ngram
Sort order:    

Sorting:   0%|          | 0/1 [00:00<?, ?files/s]

Merging complete. Final file: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4

End Time:                  2025-02-03 16:20:23.692009
Total runtime:             0:01:04.329601



### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [2]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/'
        '1gram_files/6corpus/1gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)


The file is sorted.

Processing complete.


### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime with compression:  0:07:33.662163`

In [4]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    lines_per_chunk=1000000,
    compress=True,
    overwrite=True
)

Start Time:                2025-02-03 19:27:35.164647

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/temp_chunks
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True

Created and Sorted: 3 chunks
Merged: 3 chunks


End Time:                  2025-02-03 19:28:46.067911
Total runtime:             0:01:10.903264



### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-corpus-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-corpus-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

In [6]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction',
    compress=True,
    overwrite=False,
    vocab_n=80000
)

Start Time:                2025-02-03 19:29:44.841838

Indexing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Indexed file:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-indexed.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  False

Vocabulary Info
Vocab size (top N):        80000
Match File:                /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus-vocab_list_match.txt
Lookup File:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/1gram_files/6corpus/1gram-corpus-vocab_list_lookup.jsonl



Chunking:   0%|          | 0/1125984 [00:00<?, ?lines/s]

Sorting:   0%|          | 0/12 [00:00<?, ?chunks/s]

Merging:   0%|          | 0/1125984 [00:00<?, ?lines/s]

Indexing:   0%|          | 0/1125984 [00:00<?, ?lines/s]


Indexed 1125984 lines.
Final indexed file: 1gram-corpus-indexed.jsonl.lz4
Created vocab_list_match and vocab_list_lookup files for top 80000 ngrams.

End Time:                  2025-02-03 19:30:50.254639
Total runtime:             0:01:05.412801



### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [5]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/'
        '1gram_files/6corpus/1gram-corpus-indexed.jsonl.lz4'
    ),
    start_line=1,
    end_line=5,
    parse_json=True
)

Line 1: {'ngram': 'say', 'freq_tot': 707702607, 'doc_tot': 8591543, 'freq': {'1806': 18685, '1827': 138448, '1844': 311640, '1845': 420341, '1849': 226290, '1862': 268241, '1866': 491995, '1867': 535016, '1870': 564639, '1871': 585908, '1875': 549010, '1877': 522266, '1878': 428208, '1879': 487523, '1880': 640696, '1890': 845614, '1891': 866986, '1892': 1060527, '1893': 1227248, '1894': 1149638, '1896': 1418741, '1897': 1380111, '1898': 1348630, '1899': 2375549, '1900': 2189043, '1901': 1814746, '1902': 1319918, '1904': 1411106, '1905': 1266894, '1906': 1543400, '1908': 1358684, '1909': 1402152, '1910': 1694710, '1911': 1316149, '1912': 1243268, '1913': 1149007, '1914': 1091615, '1915': 1089231, '1916': 910723, '1917': 1244965, '1918': 753382, '1920': 1226910, '1921': 984251, '1924': 689655, '1927': 925026, '1928': 900615, '1929': 948350, '1934': 767541, '1935': 856967, '1936': 793343, '1937': 827396, '1938': 758057, '1939': 700118, '1940': 750959, '1942': 639124, '1943': 558856, '1944